In [4]:
import pickle
import pandas as pd

with open("data/X_meta_use-target_sub.pickle", "rb") as f:
    X_meta = pickle.load(f)

X_train_meta, X_valid_meta, X_test_meta = X_meta
    
with open("data/X_token_use-target_sub.pickle", "rb") as f:
    X_token = pickle.load(f)

X_train_token, X_valid_token, X_test_token = X_token
    
with open("data/y_use-target_sub.pickle", "rb") as f:
    y = pickle.load(f)
    
y_train, y_valid, y_test = y

In [5]:
def concat_tokens(tokens):
    return [" ".join(x) for x in tokens]

X_train_token = concat_tokens(X_train_token)
X_valid_token = concat_tokens(X_valid_token)
X_test_token = concat_tokens(X_test_token)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

count_vectorizer = CountVectorizer(min_df=.015, max_df=.8, max_features=1000, ngram_range=[1, 3])
count_vectorizer.fit(X_train_token)
X_train_token_vec = pd.DataFrame.sparse.from_spmatrix(count_vectorizer.transform(X_train_token))
X_valid_token_vec = pd.DataFrame.sparse.from_spmatrix(count_vectorizer.transform(X_valid_token))
X_test_token_vec = pd.DataFrame.sparse.from_spmatrix(count_vectorizer.transform(X_test_token))


In [7]:
X_train = pd.concat([X_train_meta, X_train_token_vec], axis=1)
X_valid = pd.concat([X_valid_meta, X_valid_token_vec], axis=1)
X_test = pd.concat([X_test_meta, X_test_token_vec], axis=1)

y_train = pd.DataFrame(y_train, columns=['target'])
y_valid = pd.DataFrame(y_valid, columns=['target'])
y_test = pd.DataFrame(y_test, columns=['target'])

train_df = pd.concat([X_train, y_train], axis=1)
valid_df = pd.concat([X_valid, y_valid], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [8]:
from pycaret.classification import *

clf = setup(data=train_df, target="target", 
            session_id=42,
            fold=5, fold_shuffle=True, 
            fix_imbalance=True, 
            normalize=True,)

,Description,Value
0,session_id,42
1,Target,target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(16549, 387)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,377
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
lr = create_model('lr')
tuned_lr = tune_model(lr, optimize='f1')
final_lr = finalize_model(tuned_lr)
pred_lr = predict_model(final_lr, data=test_df)
pred_lr

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6831,0.7149,0.6025,0.3909,0.4742,0.2618,0.2744
1,0.7060,0.7390,0.6310,0.4207,0.5048,0.3075,0.3203
2,0.6846,0.7162,0.5916,0.3914,0.4711,0.2595,0.2707
3,0.7015,0.7372,0.6196,0.4141,0.4964,0.2960,0.3082
4,0.6893,0.7125,0.5834,0.3952,0.4712,0.2626,0.2726
Mean,0.6929,0.7240,0.6056,0.4025,0.4836,0.2775,0.2892
Std,0.0092,0.0116,0.0175,0.0125,0.0142,0.0202,0.0208


/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.7025,0.7589,0.6515,0.4185,0.5097,0.3104,0.3261


,center,age,gender,height,weight,BMI,BMI_group,is_operation,is_pain,pain_NRS,...,358,359,360,361,362,363,364,target,Label,Score
0,13,20,1,187.0,71.0,20.303698,1.0,0.0,1.0,3.0,...,0,0,1,0,0,0,0,0,0,0.6417
1,2,24,1,175.0,73.0,23.836735,1.0,0.0,1.0,2.0,...,0,0,0,1,0,1,0,0,1,0.5097
2,12,22,1,178.0,64.0,20.199470,1.0,1.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0.5513
3,12,21,1,172.0,80.0,27.041644,2.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,1,0.6884
4,14,20,1,178.0,58.0,18.305769,1.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,1,0.6630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039,11,20,1,170.0,70.0,24.221453,1.0,1.0,1.0,4.0,...,0,0,0,1,0,0,0,0,1,0.7201
2040,15,23,1,171.0,82.0,28.042817,2.0,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0.5018
2041,12,21,1,173.0,60.0,20.047446,1.0,0.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0.5962
2042,12,21,1,168.0,66.0,23.384354,1.0,0.0,1.0,4.0,...,0,0,0,0,0,0,0,0,0,0.6414


/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

In [8]:
dt = create_model('dt')
tuned_dt = tune_model(dt, optimize='f1')
final_dt = finalize_model(tuned_dt)
pred_dt = predict_model(final_dt, data=test_df)
pred_dt

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7631,0.6949,0.3439,0.5009,0.4079,0.2662,0.2735
1,0.7873,0.6879,0.2875,0.6108,0.3910,0.2818,0.3113
2,0.7813,0.6843,0.2812,0.5816,0.3791,0.2654,0.2912
3,0.7789,0.6878,0.3321,0.5577,0.4163,0.2905,0.3054
4,0.7830,0.6907,0.2994,0.5831,0.3956,0.2797,0.3028
Mean,0.7787,0.6891,0.3088,0.5668,0.3980,0.2767,0.2968
Std,0.0083,0.0035,0.0248,0.0370,0.0130,0.0097,0.0134


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.7847,0.7128,0.3258,0.5830,0.4180,0.2987,0.3177


,center,age,gender,height,weight,BMI,BMI_group,is_operation,is_pain,pain_NRS,...,358,359,360,361,362,363,364,target,Label,Score
0,13,20,1,187.0,71.0,20.303698,1.0,0.0,1.0,3.0,...,0,0,1,0,0,0,0,0,0,0.5387
1,2,24,1,175.0,73.0,23.836735,1.0,0.0,1.0,2.0,...,0,0,0,1,0,1,0,0,0,0.7615
2,12,22,1,178.0,64.0,20.199470,1.0,1.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0.5387
3,12,21,1,172.0,80.0,27.041644,2.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0.5387
4,14,20,1,178.0,58.0,18.305769,1.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0.8120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039,11,20,1,170.0,70.0,24.221453,1.0,1.0,1.0,4.0,...,0,0,0,1,0,0,0,0,0,0.7877
2040,15,23,1,171.0,82.0,28.042817,2.0,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0.5387
2041,12,21,1,173.0,60.0,20.047446,1.0,0.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0.9543
2042,12,21,1,168.0,66.0,23.384354,1.0,0.0,1.0,4.0,...,0,0,0,0,0,0,0,0,0,0.7877


In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb_param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5, 7, 10, 20]
}

xgb = XGBClassifier()
xgb_grid = GridSearchCV(xgb, param_grid=xgb_param_grid, scoring="f1", 
                        verbose=1, n_jobs=-1, cv=5)

xgb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.05, 0.01],
                         'max_depth': [3, 5, 7, 10, 20],
                         'n_estimators': [100, 300, 500]},
             scoring='f1', verbose=1)

In [7]:
params = xgb_grid.best_params_
xgb = XGBClassifier(learning_rate=params['learning_rate'],
                    max_depth=params['max_depth'], 
                    n_estimators=params['n_estimators'])

xgb.fit(X_train, y_train)
xgb_out = xgb.predict_proba(X_test)[:, 1]

In [11]:
mlp = create_model('mlp')
tuned_mlp = tune_model(mlp, optimize='f1')
final_mlp = finalize_model(tuned_mlp)
pred_mlp = predict_model(final_mlp, data=test_df)
pred_mlp

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6979,0.7111,0.5643,0.4024,0.4698,0.2667,0.2742
1,0.7689,0.7562,0.5076,0.5135,0.5106,0.3593,0.3593
2,0.7100,0.7344,0.5751,0.4193,0.4850,0.2900,0.2970
3,0.6807,0.7554,0.7023,0.4015,0.5109,0.2991,0.3249
4,0.7217,0.7397,0.5618,0.4332,0.4892,0.3023,0.3071
Mean,0.7158,0.7394,0.5822,0.4340,0.4931,0.3035,0.3125
Std,0.0298,0.0165,0.0644,0.0415,0.0158,0.0306,0.0286


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,MLP Classifier,0.7021,0.7461,0.6557,0.4184,0.5108,0.3113,0.3276


,center,age,gender,height,weight,BMI,BMI_group,is_operation,is_pain,pain_NRS,...,358,359,360,361,362,363,364,target,Label,Score
0,13,20,1,187.0,71.0,20.303698,1.0,0.0,1.0,3.0,...,0,0,1,0,0,0,0,0,0,0.5793
1,2,24,1,175.0,73.0,23.836735,1.0,0.0,1.0,2.0,...,0,0,0,1,0,1,0,0,1,0.7316
2,12,22,1,178.0,64.0,20.199470,1.0,1.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0.5514
3,12,21,1,172.0,80.0,27.041644,2.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,1,0.8643
4,14,20,1,178.0,58.0,18.305769,1.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0.5548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039,11,20,1,170.0,70.0,24.221453,1.0,1.0,1.0,4.0,...,0,0,0,1,0,0,0,0,1,0.5505
2040,15,23,1,171.0,82.0,28.042817,2.0,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0.6876
2041,12,21,1,173.0,60.0,20.047446,1.0,0.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0.7261
2042,12,21,1,168.0,66.0,23.384354,1.0,0.0,1.0,4.0,...,0,0,0,0,0,0,0,0,0,0.8384


In [37]:
res = pd.read_excel("model_performances_use-target_sub.xlsx", engine="openpyxl")
res["Score_lr"] = pred_lr["Score"].values
res['Score_dt'] = pred_dt['Score'].values
res['Score_xgboost'] = xgb_out
res['Score_mlp'] = pred_mlp['Score'].values

res.head()

,label,Score_dnn,Score_lr,Score_dt,Score_xgboost,Score_mlp
0,0,0.188580,0.5036,0.5901,0.342983,0.5531
1,0,0.394876,0.7209,0.7863,0.306893,0.7705
2,0,0.118185,0.5814,0.5558,0.189198,0.6307
3,0,0.120731,0.6142,0.5901,0.260582,0.7165
4,0,0.363556,0.6445,0.8325,0.110405,0.7084


In [38]:
res.to_excel("model_performances_use-target_sub.xlsx", index=False)